In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from mne.stats import fdr_correction
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
import random 
import subprocess
import matplotlib as mpl
from matplotlib import animation

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.2)
sns.set_style('ticks')

get_ipython().run_line_magic('config', "InlineBackend.figure_format = 'svg'")
plt.rcParams['figure.dpi'] = 200
plt.rcParams['svg.fonttype'] = 'none'

In [ ]:
# samples metadata
filtered_reads_samples_filterdis  = pd.read_pickle('.../metadata.pkl')

In [ ]:
# use human/bacterial protein abundance profile to run RF
all_uniref_human_profile = glob('hp_abun/*human_cluster_filter.pkl')
all_uniref_human_profile_df = [pd.read_pickle(x) for x in all_uniref_human_profile]

# all study names
study_name = pd.DataFrame(all_uniref_human_profile)[0].str.replace('hp_abun/','').str.replace('_human_cluster_filter.pkl','').tolist()

In [ ]:
# attach "labels" to the metagenomic samples 
def return_all_factorized():
    all_df_study = []
    for i in range(len(study_name)):
        temp = all_uniref_human_profile_df[i].reset_index().merge(filtered_reads_samples_filterdis[filtered_reads_samples_filterdis.study_name == study_name[i]][['sample_id','study_condition']].dropna(), right_on = 'sample_id', left_on = 'index').drop('index', axis = 1).rename(columns = {'study_condition':'y'})
        temp.index = temp.sample_id
        temp.drop('sample_id', axis = 1, inplace = True)
        temp.loc[temp['y']!='control', 'y'] = 1
        temp.loc[temp['y']=='control', 'y'] = 0
        temp['y'] = temp['y'].astype('int')
        temp = temp.reset_index(drop = True)
        all_df_study.append(temp)
    return all_df_study

all_uniref_human_profile_df_fac = return_all_factorized()

In [ ]:
plt.rcParams.update({'figure.max_open_warning': 0})
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, average_precision_score, precision_score, recall_score, f1_score, roc_auc_score
from scipy import interp
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV

def run_rf_gridsearch(data, study_name):

    X = data.iloc[:,:-1]
    y = data['y']
        
    rfc = RandomForestClassifier(class_weight = 'balanced')
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 100, stop = 5000, num = 8)]
    # Number of features to consider at every split
    max_features = ['auto']
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 200, num = 5)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True]
    
    # Create the random grid
    random_grid = {'n_estimators': n_estimators,
                   'max_features': max_features,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf,
                   'bootstrap': bootstrap}

    grid_search = GridSearchCV(rfc, random_grid, cv = 5, verbose=3, n_jobs = 30)
    grid_search.fit(X, y)
    
    return grid_search.best_params_

In [ ]:
# grid search
best_params_all = []
for i in range(len(study_name)):
    best_params_all.append(run_rf_gridsearch(all_uniref_human_profile_df_fac[i],study_name[i]))

In [ ]:
# save 
best_para = pd.DataFrame(best_params_all)
best_para['study_name'] = study_name
best_para.to_pickle('RF_final/best_hyperparam.pkl')